In [ ]:
## このプログラムは「Digit Recognizer」
## において作成したノートブックにおいて動作します

# データを用意する

import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from tensorflow.keras.utils import to_categorical

# train.csvを読み込んでpandasのDataFrameに格納
train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
train_x = train.drop(['label'], axis=1) # trainから画像データを抽出
train_y = train['label']                # trainから正解ラベルを抽出
test_x = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

# trainのデータを学習データとテストデータに分ける。
kf = KFold(n_splits=4, shuffle=True, random_state=123)
tr_idx, va_idx = list(kf.split(train_x))[0]
tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

# 画像のピクセル値を255.0で割って0～1.0の範囲にしてnumpy.arrayに変換
tr_x, va_x = np.array(tr_x / 255.0), np.array(va_x / 255.0)

# 画像データの2階テンソルを
# (高さ = 28px, 幅 = 28px , チャンネル = 1)の
# 3階テンソルに変換
# グレースケールのためチャンネルは1。
tr_x = tr_x.reshape(-1,28,28,1)
va_x = va_x.reshape(-1,28,28,1)

# 正解ラベルをone-hot表現に変換
tr_y = to_categorical(tr_y, 10)
va_y = to_categorical(va_y, 10)

In [ ]:
# モデルを生成する
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten # core layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D    # convolution layers

# Sequentialオブジェクトを生成
model = Sequential()

# 第1層:畳み込み層
model.add(Conv2D(filters=32,
                 kernel_size=(5,5),
                 padding='same', 
                 activation='relu',
                 input_shape=(28,28,1)))

# 第2層:畳み込み層
model.add(Conv2D(filters = 64,
                 kernel_size = (7,7),
                 padding='same', 
                 activation='relu'))

# 第3層:プーリング層
model.add(MaxPooling2D(pool_size=(2,2)))

# ドロップアウト
model.add(Dropout(0.5))

# 第4層:畳み込み層
model.add(Conv2D(filters=64,
                 kernel_size=(5,5),
                 padding='same', 
                 activation='relu'))

# 第5層:畳み込み層
model.add(Conv2D(filters = 32,
                 kernel_size = (3,3),
                 padding='same', 
                 activation='relu'))

# 第6層:プーリング層
model.add(MaxPooling2D(pool_size=(2,2)))

# ドロップアウト
model.add(Dropout(0.55))

# Flatten層
model.add(Flatten())

# 第7層：全結合層
model.add(Dense(700, activation='relu'))
model.add(Dropout(0.3))

# 第8層：全結合層
model.add(Dense(150, activation='relu'))
model.add(Dropout(0.35))

# 第10層：出力層
model.add(Dense(10, activation = "softmax"))

# モデルのコンパイル
# オプティマイザーはAdam
momentum = 0.5
model.compile(loss="categorical_crossentropy",
              optimizer='adam',
              metrics=["accuracy"])

# モデルの構造を出力。
model.summary()

In [ ]:
# ミニバッチのサイズ
batch_size = 100
# 学習回数
epochs = 20

# 学習を行う
history = model.fit(tr_x, tr_y,                 # 訓練データ
                    batch_size=batch_size,      # ミニバッチのサイズ
                    epochs=epochs,              # 学習回数
                    verbose=1,                  # 学習の進捗状況を出力する
                    validation_data=(va_x,va_y) # テストデータ
                    )


In [ ]:
# 損失と精度の推移をグラフにする

%matplotlib inline
import matplotlib.pyplot as plt

# プロット図のサイズを設定
plt.ﬁgure(ﬁgsize=(15, 6))
# プロット図を縮小して図の間のスペースを空ける
plt.subplots_adjust(wspace=0.2)

# 1×2のグリッドの左(1,2,1)の領域にプロット
plt.subplot(1, 2, 1)
# 訓練データの損失(誤り率)をプロット
plt.plot(history.history['loss'],
         label='training',
         color='black')
# テストデータの損失(誤り率)をプロット
plt.plot(history.history['val_loss'],
         label='test',
         color='red')
plt.ylim(0, 1)       # y軸の範囲
plt.legend()         # 凡例を表示
plt.grid()           # グリッド表示
plt.xlabel('epoch')  # x軸ラベル
plt.ylabel('loss')   # y軸ラベル

# 1×2のグリッドの右(1,2,21)の領域にプロット
plt.subplot(1, 2, 2)
# 訓練データの正解率をプロット
plt.plot(history.history['accuracy'],
         label='training',
         color='black')
# テストデータの正解率をプロット
plt.plot(history.history['val_accuracy'],
         label='test',
         color='red')
plt.ylim(0.5, 1)     # y軸の範囲
plt.legend()         # 凡例を表示
plt.grid()           # グリッド表示
plt.xlabel('epoch')  # x軸ラベル
plt.ylabel('acc')    # y軸ラベル
plt.show()